In [1]:
import sys
sys.path.insert(1, '../..')

import torch
import torch.nn as nn
import random
import pandas as pd
import numpy as np
import time

from library.evaluation import ConfusionMatrix

random.seed(33)

dataset_name = "Phemernr2_CrossVal"
unique_name = "RoBERTa"

In [2]:
vectors = np.loadtxt("../../data/processed/vectors/Phemernr2_RoBERTa_vectors.txt", delimiter=",")
first = vectors[0]
vectors.shape

(6425, 768)

In [3]:
data = pd.read_csv("../../data/processed/phemernr2_dataset_with_tvt.csv", lineterminator="\n")
data.head()

,tweet_id,tweet_text,label,label2,topic,tvt,cv_fold,tt
0,552833795142209536,The East London Mosque would like to offer its...,non-rumours,non-rumours,charliehebdo-all-rnr-threads,test,2,test
1,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,rumours,true,germanwings-crash-all-rnr-threads,training,3,training
2,552798891994009601,Reports that two of the dead in the #CharlieHe...,rumours,true,charliehebdo-all-rnr-threads,test,2,test
3,576790814942236672,After #Putin disappeared Russian TV no longer ...,non-rumours,non-rumours,putinmissing-all-rnr-threads,test,2,test
4,499678822598340608,Saw #Ferguson for myself. #justiceformichaelbr...,non-rumours,non-rumours,ferguson-all-rnr-threads,training,3,training


In [4]:
labels_str = data['label2'].unique().tolist()
labels_str

['non-rumours', 'true', 'unverified', 'false']

In [5]:
labels = []
for i, d in data.iterrows():
    lab = labels_str.index(d['label2'])
    labels.append(lab)
labels[:10]

[0, 1, 1, 0, 0, 0, 0, 2, 0, 0]

In [6]:
test_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] == 0])
test_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] == 0])

In [7]:
print(test_vectors.shape)
print(test_labels.shape)

(630, 768)
(630,)


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes):
        super(BasicBlock, self).__init__()
        self.lin1 = nn.Linear(in_planes, planes)
        self.bn1 = nn.BatchNorm1d(planes)
        self.lin2 = nn.Linear(planes, planes)
        self.bn2 = nn.BatchNorm1d(planes)

        self.shortcut = nn.Sequential()
        if in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Linear(in_planes, planes),
                nn.BatchNorm1d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.lin1(x)), 0.1)
        out = self.bn2(self.lin2(out))
        out += self.shortcut(x)
        out = F.leaky_relu(out, 0.1)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes):
        super(Bottleneck, self).__init__()
        self.lin1 = nn.Linear(in_planes, planes)
        self.bn1 = nn.BatchNorm1d(planes)
        self.lin2 = nn.Linear(planes, planes)
        self.bn2 = nn.BatchNorm1d(planes)
        self.lin3 = nn.Linear(planes, planes)
        self.bn3 = nn.BatchNorm1d(planes)

        self.shortcut = nn.Sequential()
        if in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Linear(in_planes, planes),
                nn.BatchNorm1d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.lin1(x)), 0.1)
        out = F.leaky_relu(self.bn2(self.lin2(out)), 0.1)
        out = self.bn3(self.lin3(out))
        out += self.shortcut(x)
        out = F.leaky_relu(out, 0.1)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, n_input=768, num_classes=1):
        super(ResNet, self).__init__()
        self.in_planes = 512

        self.lin1 = nn.Linear(n_input, self.in_planes)
        self.bn1 = nn.BatchNorm1d(self.in_planes)
        self.layer1 = self._make_layer(block, 512, num_blocks[0])
        self.layer2 = self._make_layer(block, 256, num_blocks[1])
        self.layer3 = self._make_layer(block, 128, num_blocks[2])
        self.layer4 = self._make_layer(block, 64, num_blocks[3])
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, block, planes, num_blocks):
        strides = [1] * num_blocks
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.lin1(x)), 0.1)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet10(n_input=768, n_output=1, block=BasicBlock):
    return ResNet(block, [1, 1, 1, 1], n_input, n_output)

    
def ResNet18(n_input=768, n_output=1, block=BasicBlock):
    return ResNet(block, [2, 2, 2, 2], n_input, n_output)


def ResNet34(n_input=768, n_output=1, block=BasicBlock):
    return ResNet(block, [3, 4, 6, 3], n_input, n_output)


def ResNet50(n_input=768, n_output=1, block=Bottleneck):
    return ResNet(block, [3, 4, 6, 3], n_input, n_output)


def ResNet101(n_input=768, n_output=1, block=Bottleneck):
    return ResNet(block, [3, 4, 23, 3], n_input, n_output)


def ResNet152(n_input=768, n_output=1, block=Bottleneck):
    return ResNet(block, [3, 8, 36, 3], n_input, n_output)

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class CNNBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(CNNBasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)), 0.1)
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.leaky_relu(out, 0.1)
        return out


class CNNBottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes):
        super(CNNBottleneck, self).__init__()
        self.lin1 = nn.Linear(in_planes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.lin2 = nn.Linear(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.lin3 = nn.Linear(planes, planes)
        self.bn3 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Linear(in_planes, planes),
                nn.BatchNorm1d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.lin1(x)), 0.1)
        out = F.leaky_relu(self.bn2(self.lin2(out)), 0.1)
        out = self.bn3(self.lin3(out))
        out += self.shortcut(x)
        out = F.leaky_relu(out, 0.1)
        return out


class CNNResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=1):
        super(CNNResNet, self).__init__()
        self.in_planes = 24

        self.conv1 = nn.Conv2d(1, 24, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(24)
        self.layer1 = self._make_layer(block, 24, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 48, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 64, num_blocks[3], stride=2)
        self.linear = nn.Linear(64 * 24 * 32, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)), 0.1)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def CNNResNet10(n_output: int):
    return CNNResNet(CNNBasicBlock, [1, 1, 1, 1], n_output)

    
def CNNResNet18(n_output: int):
    return CNNResNet(CNNBasicBlock, [2, 2, 2, 2], n_output)


def CNNResNet34(n_output: int):
    return CNNResNet(CNNBasicBlock, [3, 4, 6, 3], n_output)


def CNNResNet50(n_output: int):
    return CNNResNet(CNNBottleneck, [3, 4, 6, 3], n_output)


def CNNResNet101(n_output: int):
    return CNNResNet(CNNBottleneck, [3, 4, 23, 3], n_output)


def CNNResNet152(n_output: int):
    return CNNResNet(CNNBottleneck, [3, 8, 36, 3], n_output)

In [10]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import os
from typing import Callable


class ResNetClassifier(nn.Module):
    def __init__(self,
        model,
        n_input: int,
        n_output: int = 1,
        criterion: Callable = nn.BCELoss,
        n_features: int = 4,
        lr: float = 0.0002,
        beta1: float = 0.5,
        device: str = None,
        model_type: str = "mlp"
    ):
        super(ResNetClassifier, self).__init__()
        self.model = model
        self.model_type = model_type
        self.criterion = criterion()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr, betas=(beta1, 0.999))
        
        if not device or device not in ['cpu', 'cuda']:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            self.device = device

        self.model = self.model.to(self.device)
        if self.device == 'cuda':
            self.model = torch.nn.DataParallel(self.model)
            cudnn.benchmark = True

    def forward(self, input):
        x = self.model(input)
        return x
    
    def load_pretrained(self, filepath: str, key: str = "net", is_parallel: bool = False):
        checkpoint = torch.load(filepath)
        if is_parallel:
            self.model = torch.nn.DataParallel(self.model)
        self.model.load_state_dict(checkpoint[key], strict=False)
    
    def save_model(self, saves: str):
        print(f"Saving model...")
        state = {
            'net': self.model.state_dict()
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f"../../data/models/{saves}.pth")
    
    def train_eval(self,
        train_x, train_y,
        test_x, test_y,
        n_iter: int = 100,
        batch_size: int = 128,
        saves: str = None
    ):
        trainset = torch.utils.data.TensorDataset(train_x, train_y) # create your datset
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size) # create your dataloader

        testset = torch.utils.data.TensorDataset(test_x, test_y) # create your datset
        testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size) # create your dataloader

        train_accs = []
        train_losses = []
        test_accs = []
        test_losses = []

        print(f"Using {self.device}")
        best_acc = 0
        current_loss = 1000
        best_test_acc = 0
        epoch = 0
        start_time = time.time()
        results = {}
        while True:
            epoch += 1
            self.model.train()
            train_loss = 0
            correct = 0
            total = 0
            for batch_idx, (inputs, targets) in enumerate(trainloader):
                self.model.zero_grad()
                inputs, targets = inputs.to(self.device), targets.to(self.device)

                if self.model_type == "cnn":
                    outputs = self.model(inputs.unsqueeze(1))
                elif self.model_type == "mlp":
                    outputs = self.model(inputs)

                loss = self.criterion(outputs, targets.long())
                loss.backward()
                self.optimizer.step()

                train_loss += loss.item()
                total += targets.size(0)
#                 for i, output in enumerate(outputs.tolist()):
#                     if targets[i,0].tolist() == round(output[0]):
#                         correct += 1

#             train_acc = round(100*correct/total, 4)
#             train_accs.append(train_acc)
            train_losses.append(train_loss)

            self.model.eval()
            test_loss = 0
            test_acc = 0
            with torch.no_grad():
                inputs, targets = test_x.to(self.device), test_y.to(self.device)
                
                if self.model_type == 'mlp':
                    outputs = self.model(inputs)
                else:
#                     outputs = self.model(inputs.reshape(inputs.shape[0], 1, 24, 32))
                    outputs = self.model(inputs.unsqueeze(1))

                loss = self.criterion(outputs, targets.long())
                test_loss += loss.item()
                
                if self.model_type == 'mlp':
                    preds = self.predict(test_x)
                else:
                    preds = self.predict(test_x.reshape(test_x.shape[0], 1, 24, 32))
                conf_mat = ConfusionMatrix(
                    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_y]),
                    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds.cpu().numpy()]),
                    binary=False
                )
                conf_mat.evaluate(logs=False)
                test_acc = conf_mat.accuracy

            test_losses.append(test_loss)
            
#             if (epoch) % round(n_iter/20) == 0:
#                 print(f"-- Epoch {epoch}, Train Loss : {train_loss}, Test Loss : {test_loss}")

            # Save checkpoint.
#             if saves and test_loss < best_loss:
#                 print(f"Saving after new best loss : {test_loss}")
#                 best_loss = test_loss
            if saves and test_acc > best_test_acc:
#                 print(f"Saving after new best accuracy : {test_acc}")
                best_test_acc = test_acc

                state = {
                    'net': self.model.state_dict(),
                }
                if not os.path.isdir('models'):
                    os.mkdir('models')
                torch.save(state, f"../../data/models/{saves}.pth")
            
            if epoch >= n_iter:
                break

#         # visualizing accuracy over epoch
#         fig, ax2 = plt.subplots(1)
#         plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=1.5, wspace=0.4)

#         ax2.plot([i for i in range(len(train_losses))], train_losses, c='b', marker="o", label='Train Loss')
#         ax2.plot([i for i in range(len(test_losses))], test_losses, c='r', marker="o", label='Test Loss')
#         ax2.set_ylabel('Loss')
#         ax2.set_xlabel('Epoch')
#         ax2.set_xlim(0, len(train_losses))
#         ax2.set_ylim(min([min(train_losses), min(test_losses)])*0.1, max([max(train_losses), max(test_losses)]))
#         ax2.title.set_text(f"Loss over time (epoch)")
#         ax2.legend(loc='lower right')

#         plt.show()
    
    def predict(self, input_x):
        self.model.eval()
        with torch.no_grad():
            preds = self.model(torch.Tensor(input_x))
            preds = torch.log_softmax(preds, dim = 1)
            _, preds = torch.max(preds, dim = 1)
            return preds

In [11]:
print("3-Fold Cross Validation with ResNet10 CNN")

folds = [1, 2, 3]
for fold in folds:
    start = time.time()
    val_folds = [fold]
    train_folds = folds.copy()
    train_folds.remove(fold)

    train_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] in train_folds])
    val_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] in val_folds])

    train_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] in train_folds])
    val_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] in val_folds])

    print(f"Fold-{fold} Cross Validation")
    model_name = f"{dataset_name}_ResNet10_CNN_{unique_name}_fold-{fold}"
    print(f"Multiclass Classification {model_name}")
    model = ResNetClassifier(CNNResNet10(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")
    model.train_eval(torch.Tensor(train_vectors.reshape(train_vectors.shape[0], 24, 32)),
                    torch.Tensor(train_labels),
                    torch.Tensor(val_vectors.reshape(val_vectors.shape[0], 24, 32)),
                    torch.Tensor(val_labels),
                    saves=model_name,
                    n_iter=1000,
                    batch_size=1024)

    model.load_pretrained(f"../../data/models/{model_name}.pth")

    print(f"\n-------- Fold-{fold} Results --------")
    preds = model.predict(val_vectors.reshape(val_vectors.shape[0], 1, 24, 32))

    preds = preds.cpu().numpy()

    conf_mat = ConfusionMatrix(
        labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in val_labels]),
        predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
        binary=False,
        model_name=model_name
    )
    conf_mat.evaluate(classes=labels_str)
    
    print(f"-- Execution Time : {round(time.time() - start, 2)} seconds")
    print(f"-------- Fold-{fold} End --------\n")

3-Fold Cross Validation with ResNet10 CNN
Fold-1 Cross Validation
Multiclass Classification Phemernr2_CrossVal_ResNet10_CNN_RoBERTa_fold-1
Using cuda

-------- Fold-1 Results --------
1914 vs 1914
Multi Class Evaluation

Class non-rumours Evaluation
- Precision : 85.594 %
- Recall : 91.109 %
- F1 : 0.88266

Class true Evaluation
- Precision : 65.528 %
- Recall : 64.724 %
- F1 : 0.65123

Class unverified Evaluation
- Precision : 61.871 %
- Recall : 45.989 %
- F1 : 0.52761

Class false Evaluation
- Precision : 77.027 %
- Recall : 65.143 %
- F1 : 0.70588

Combined Evaluation
- Accuracy : 79.833 %
- Precision : 72.505 %
- Recall : 66.741 %
- F1 : 0.69504

- Average Confidence : 100.0 %
Model, Combined,,,,non-rumours,,,true,,,unverified,,,false,,,
Phemernr2_CrossVal_ResNet10_CNN_RoBERTa_fold-1, 79.833, 72.505, 66.741, 0.69504, 85.594, 91.109, 0.88266, 65.528, 64.724, 0.65123, 61.871, 45.989, 0.52761, 77.027, 65.143, 0.70588, 
-- Execution Time : 623.05 seconds
-------- Fold-1 End --------



In [12]:
fold_n = 1
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_ResNet10_CNN_{unique_name}_fold-{fold_n}"
model = ResNetClassifier(CNNResNet10(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors.reshape(test_vectors.shape[0], 1, 24, 32))

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-1

-------- Testing Results --------
630 vs 630
Multi Class Evaluation

Class non-rumours Evaluation
- Precision : 84.069 %
- Recall : 90.501 %
- F1 : 0.87166

Class true Evaluation
- Precision : 66.071 %
- Recall : 67.89 %
- F1 : 0.66968

Class unverified Evaluation
- Precision : 65.0 %
- Recall : 52.0 %
- F1 : 0.57778

Class false Evaluation
- Precision : 68.0 %
- Recall : 50.746 %
- F1 : 0.5812

Combined Evaluation
- Accuracy : 77.778 %
- Precision : 70.785 %
- Recall : 65.284 %
- F1 : 0.67923

- Average Confidence : 100.0 %
Model, Combined,,,,non-rumours,,,true,,,unverified,,,false,,,
Phemernr2_CrossVal_ResNet10_CNN_RoBERTa_fold-1, 77.778, 70.785, 65.284, 0.67923, 84.069, 90.501, 0.87166, 66.071, 67.89, 0.66968, 65.0, 52.0, 0.57778, 68.0, 50.746, 0.5812, 
-------- Testing End --------



In [13]:
fold_n = 2
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_ResNet10_CNN_{unique_name}_fold-{fold_n}"
model = ResNetClassifier(CNNResNet10(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors.reshape(test_vectors.shape[0], 1, 24, 32))

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-2

-------- Testing Results --------
630 vs 630
Multi Class Evaluation

Class non-rumours Evaluation
- Precision : 81.336 %
- Recall : 93.14 %
- F1 : 0.86839

Class true Evaluation
- Precision : 68.317 %
- Recall : 63.303 %
- F1 : 0.65714

Class unverified Evaluation
- Precision : 79.07 %
- Recall : 45.333 %
- F1 : 0.57627

Class false Evaluation
- Precision : 76.923 %
- Recall : 59.701 %
- F1 : 0.67227

Combined Evaluation
- Accuracy : 78.73 %
- Precision : 76.412 %
- Recall : 65.369 %
- F1 : 0.7046

- Average Confidence : 100.0 %
Model, Combined,,,,non-rumours,,,true,,,unverified,,,false,,,
Phemernr2_CrossVal_ResNet10_CNN_RoBERTa_fold-2, 78.73, 76.412, 65.369, 0.7046, 81.336, 93.14, 0.86839, 68.317, 63.303, 0.65714, 79.07, 45.333, 0.57627, 76.923, 59.701, 0.67227, 
-------- Testing End --------



In [14]:
fold_n = 3
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_ResNet10_CNN_{unique_name}_fold-{fold_n}"
model = ResNetClassifier(CNNResNet10(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors.reshape(test_vectors.shape[0], 1, 24, 32))

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-3

-------- Testing Results --------
630 vs 630
Multi Class Evaluation

Class non-rumours Evaluation
- Precision : 86.198 %
- Recall : 87.335 %
- F1 : 0.86763

Class true Evaluation
- Precision : 60.294 %
- Recall : 75.229 %
- F1 : 0.66939

Class unverified Evaluation
- Precision : 57.143 %
- Recall : 42.667 %
- F1 : 0.48855

Class false Evaluation
- Precision : 70.37 %
- Recall : 56.716 %
- F1 : 0.6281

Combined Evaluation
- Accuracy : 76.667 %
- Precision : 68.501 %
- Recall : 65.487 %
- F1 : 0.6696

- Average Confidence : 100.0 %
Model, Combined,,,,non-rumours,,,true,,,unverified,,,false,,,
Phemernr2_CrossVal_ResNet10_CNN_RoBERTa_fold-3, 76.667, 68.501, 65.487, 0.6696, 86.198, 87.335, 0.86763, 60.294, 75.229, 0.66939, 57.143, 42.667, 0.48855, 70.37, 56.716, 0.6281, 
-------- Testing End --------



In [15]:
print("3-Fold Cross Validation with ResNet18 CNN")

folds = [1, 2, 3]
for fold in folds:
    start = time.time()
    val_folds = [fold]
    train_folds = folds.copy()
    train_folds.remove(fold)

    train_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] in train_folds])
    val_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] in val_folds])

    train_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] in train_folds])
    val_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] in val_folds])

    print(f"Fold-{fold} Cross Validation")
    model_name = f"{dataset_name}_ResNet18_CNN_{unique_name}_fold-{fold}"
    print(f"Multiclass Classification {model_name}")
    model = ResNetClassifier(CNNResNet18(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")
    model.train_eval(torch.Tensor(train_vectors.reshape(train_vectors.shape[0], 24, 32)),
                    torch.Tensor(train_labels),
                    torch.Tensor(val_vectors.reshape(val_vectors.shape[0], 24, 32)),
                    torch.Tensor(val_labels),
                    saves=model_name,
                    n_iter=1000,
                    batch_size=1024)

    model.load_pretrained(f"../../data/models/{model_name}.pth")

    print(f"\n-------- Fold-{fold} Results --------")
    preds = model.predict(val_vectors.reshape(val_vectors.shape[0], 1, 24, 32))

    preds = preds.cpu().numpy()

    conf_mat = ConfusionMatrix(
        labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in val_labels]),
        predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
        binary=False,
        model_name=model_name
    )
    conf_mat.evaluate(classes=labels_str)
    
    print(f"-- Execution Time : {round(time.time() - start, 2)} seconds")
    print(f"-------- Fold-{fold} End --------\n")

3-Fold Cross Validation with ResNet18 CNN
Fold-1 Cross Validation
Multiclass Classification Phemernr2_CrossVal_ResNet18_CNN_RoBERTa_fold-1
Using cuda

-------- Fold-1 Results --------
1914 vs 1914
Multi Class Evaluation

Class non-rumours Evaluation
- Precision : 82.577 %
- Recall : 93.556 %
- F1 : 0.87725

Class true Evaluation
- Precision : 74.167 %
- Recall : 54.601 %
- F1 : 0.62898

Class unverified Evaluation
- Precision : 56.296 %
- Recall : 40.642 %
- F1 : 0.47205

Class false Evaluation
- Precision : 75.333 %
- Recall : 64.571 %
- F1 : 0.69538

Combined Evaluation
- Accuracy : 79.101 %
- Precision : 72.093 %
- Recall : 63.343 %
- F1 : 0.67435

- Average Confidence : 100.0 %
Model, Combined,,,,non-rumours,,,true,,,unverified,,,false,,,
Phemernr2_CrossVal_ResNet18_CNN_RoBERTa_fold-1, 79.101, 72.093, 63.343, 0.67435, 82.577, 93.556, 0.87725, 74.167, 54.601, 0.62898, 56.296, 40.642, 0.47205, 75.333, 64.571, 0.69538, 
-- Execution Time : 1127.55 seconds
-------- Fold-1 End --------


In [16]:
fold_n = 1
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_ResNet18_CNN_{unique_name}_fold-{fold_n}"
model = ResNetClassifier(CNNResNet18(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors.reshape(test_vectors.shape[0], 1, 24, 32))

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-1

-------- Testing Results --------
630 vs 630
Multi Class Evaluation

Class non-rumours Evaluation
- Precision : 79.333 %
- Recall : 94.195 %
- F1 : 0.86128

Class true Evaluation
- Precision : 75.641 %
- Recall : 54.128 %
- F1 : 0.63102

Class unverified Evaluation
- Precision : 67.347 %
- Recall : 44.0 %
- F1 : 0.53226

Class false Evaluation
- Precision : 67.925 %
- Recall : 53.731 %
- F1 : 0.6

Combined Evaluation
- Accuracy : 76.984 %
- Precision : 72.561 %
- Recall : 61.514 %
- F1 : 0.66582

- Average Confidence : 100.0 %
Model, Combined,,,,non-rumours,,,true,,,unverified,,,false,,,
Phemernr2_CrossVal_ResNet18_CNN_RoBERTa_fold-1, 76.984, 72.561, 61.514, 0.66582, 79.333, 94.195, 0.86128, 75.641, 54.128, 0.63102, 67.347, 44.0, 0.53226, 67.925, 53.731, 0.6, 
-------- Testing End --------



In [17]:
fold_n = 2
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_ResNet18_CNN_{unique_name}_fold-{fold_n}"
model = ResNetClassifier(CNNResNet18(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors.reshape(test_vectors.shape[0], 1, 24, 32))

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-2

-------- Testing Results --------
630 vs 630
Multi Class Evaluation

Class non-rumours Evaluation
- Precision : 80.876 %
- Recall : 92.612 %
- F1 : 0.86347

Class true Evaluation
- Precision : 69.369 %
- Recall : 70.642 %
- F1 : 0.7

Class unverified Evaluation
- Precision : 80.0 %
- Recall : 42.667 %
- F1 : 0.55652

Class false Evaluation
- Precision : 75.556 %
- Recall : 50.746 %
- F1 : 0.60714

Combined Evaluation
- Accuracy : 78.413 %
- Precision : 76.45 %
- Recall : 64.167 %
- F1 : 0.69772

- Average Confidence : 100.0 %
Model, Combined,,,,non-rumours,,,true,,,unverified,,,false,,,
Phemernr2_CrossVal_ResNet18_CNN_RoBERTa_fold-2, 78.413, 76.45, 64.167, 0.69772, 80.876, 92.612, 0.86347, 69.369, 70.642, 0.7, 80.0, 42.667, 0.55652, 75.556, 50.746, 0.60714, 
-------- Testing End --------



In [18]:
fold_n = 3
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_ResNet18_CNN_{unique_name}_fold-{fold_n}"
model = ResNetClassifier(CNNResNet18(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors.reshape(test_vectors.shape[0], 1, 24, 32))

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-3

-------- Testing Results --------
630 vs 630
Multi Class Evaluation

Class non-rumours Evaluation
- Precision : 80.841 %
- Recall : 91.293 %
- F1 : 0.8575

Class true Evaluation
- Precision : 60.748 %
- Recall : 59.633 %
- F1 : 0.60185

Class unverified Evaluation
- Precision : 60.377 %
- Recall : 42.667 %
- F1 : 0.5

Class false Evaluation
- Precision : 78.571 %
- Recall : 49.254 %
- F1 : 0.6055

Combined Evaluation
- Accuracy : 75.556 %
- Precision : 70.134 %
- Recall : 60.712 %
- F1 : 0.65084

- Average Confidence : 100.0 %
Model, Combined,,,,non-rumours,,,true,,,unverified,,,false,,,
Phemernr2_CrossVal_ResNet18_CNN_RoBERTa_fold-3, 75.556, 70.134, 60.712, 0.65084, 80.841, 91.293, 0.8575, 60.748, 59.633, 0.60185, 60.377, 42.667, 0.5, 78.571, 49.254, 0.6055, 
-------- Testing End --------

